In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [3]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

# Data download
# ==============================================================================
url = (
       'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
       'data/simulated_items_sales.csv'
)
data = pd.read_csv(url, sep=',')

# Data preparation
# ==============================================================================
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('D')
data = data.sort_index()
exog = data.copy()
exog.columns = [f'exog_{i}' for i in range(exog.shape[1])]

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
exog_train = exog.loc[:end_train, :].copy()
exog_test  = exog.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2014-07-15 00:00:00   (n=927)
Test dates  : 2014-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=170)


In [4]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal_category',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None,
                 #fit_kwargs={'categorical_feature':'auto'}
             )

results = forecaster._create_train_X_y(series={'item_1': data_train['item_1'], 'item_2': data_train['item_2'].head(50)})

In [5]:
results

(                lag_1      lag_2      lag_3      lag_4      lag_5      lag_6  \
 date                                                                           
 2012-01-25  28.018830  23.981037  20.794986  22.503533  24.018768  24.772249   
 2012-01-26  28.747482  28.018830  23.981037  20.794986  22.503533  24.018768   
 2012-01-27  23.908368  28.747482  28.018830  23.981037  20.794986  22.503533   
 2012-01-28  21.423930  23.908368  28.747482  28.018830  23.981037  20.794986   
 2012-01-29  24.786455  21.423930  23.908368  28.747482  28.018830  23.981037   
 ...               ...        ...        ...        ...        ...        ...   
 2012-02-15  19.903125  19.239583  18.446875  19.858333  20.844792  17.282292   
 2012-02-16  22.970833  19.903125  19.239583  18.446875  19.858333  20.844792   
 2012-02-17  28.195833  22.970833  19.903125  19.239583  18.446875  19.858333   
 2012-02-18  20.221875  28.195833  22.970833  19.903125  19.239583  18.446875   
 2012-02-19  19.176042  20.2

In [6]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal_category',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None,
                 #fit_kwargs={'categorical_feature':'auto'}
             )

forecaster.fit(series=data_train, exog=exog_train)


print(forecaster.encoding_mapping)

# Show categorical features if present
cat_index = forecaster.regressor.booster_.params.get('categorical_column')
if cat_index is not None:
    features = forecaster.regressor.booster_.feature_name()
    print([features[i] for i in cat_index])

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster._create_train_X_y(data_train, exog_train)
print(X_train.dtypes)

forecaster.predict(steps=10, exog=exog_test.head(10))

{'item_1': 0, 'item_2': 1, 'item_3': 2}
['_level_skforecast']


ValueError: too many values to unpack (expected 6)

In [ ]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None,
                 #fit_kwargs={'categorical_feature':'auto'}
             )

forecaster.fit(series=data_train)


print(forecaster.encoding_mapping)

# Show categorical features if present
cat_index = forecaster.regressor.booster_.params.get('categorical_column')
if cat_index is not None:
    features = forecaster.regressor.booster_.feature_name()
    print([features[i] for i in cat_index])

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
print(X_train.dtypes)

forecaster.predict(steps=10)

{'item_1': 0, 'item_2': 1, 'item_3': 2}
lag_1                float64
lag_2                float64
lag_3                float64
lag_4                float64
lag_5                float64
lag_6                float64
lag_7                float64
lag_8                float64
lag_9                float64
lag_10               float64
lag_11               float64
lag_12               float64
lag_13               float64
lag_14               float64
lag_15               float64
lag_16               float64
lag_17               float64
lag_18               float64
lag_19               float64
lag_20               float64
lag_21               float64
lag_22               float64
lag_23               float64
lag_24               float64
_level_skforecast      int64
dtype: object


,item_1,item_2,item_3
2014-07-16,25.906323,10.522491,12.034587
2014-07-17,25.807194,10.623789,10.503966
2014-07-18,25.127355,11.299802,12.206434
2014-07-19,23.902609,11.441606,12.618740
2014-07-20,21.660527,11.658107,12.148873
2014-07-21,22.756076,11.377895,10.666930
2014-07-22,25.064381,10.869464,10.115581
2014-07-23,25.070926,12.064482,11.608842
2014-07-24,25.149565,10.882162,11.782225
2014-07-25,25.121728,10.755604,10.307792


In [ ]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'onehot',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None,
                 #fit_kwargs={'categorical_feature':'auto'}
             )

forecaster.fit(series=data_train)


print(forecaster.encoding_mapping)

# Show categorical features if present
cat_index = forecaster.regressor.booster_.params.get('categorical_column')
if cat_index is not None:
    features = forecaster.regressor.booster_.feature_name()
    print([features[i] for i in cat_index])

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
print(X_train.dtypes)

forecaster.predict(steps=10)

{'item_1': 0, 'item_2': 1, 'item_3': 2}
lag_1     float64
lag_2     float64
lag_3     float64
lag_4     float64
lag_5     float64
lag_6     float64
lag_7     float64
lag_8     float64
lag_9     float64
lag_10    float64
lag_11    float64
lag_12    float64
lag_13    float64
lag_14    float64
lag_15    float64
lag_16    float64
lag_17    float64
lag_18    float64
lag_19    float64
lag_20    float64
lag_21    float64
lag_22    float64
lag_23    float64
lag_24    float64
item_1      int32
item_2      int32
item_3      int32
dtype: object


,item_1,item_2,item_3
2014-07-16,25.860322,10.589852,11.821867
2014-07-17,25.710671,11.249642,10.875323
2014-07-18,25.254060,11.214945,12.383875
2014-07-19,24.135732,11.392770,11.752806
2014-07-20,21.717562,11.092514,11.108611
2014-07-21,22.869126,11.367802,10.028264
2014-07-22,25.297145,10.610384,9.839174
2014-07-23,25.480385,12.443352,12.550434
2014-07-24,25.524580,11.563970,11.957162
2014-07-25,25.376043,11.425697,10.186801


In [ ]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = HistGradientBoostingRegressor(random_state=123, categorical_features=['_level_skforecast']),
                 lags               = 24,
                 encoding           = 'ordinal_category',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None
             )

forecaster.fit(series=data_train)


print(forecaster.encoding_mapping)

# Show categorical features if present
cat_index = forecaster.regressor.is_categorical_
if cat_index is not None:
    features = forecaster.regressor.feature_names_in_
    print(features[cat_index])

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
print(X_train.dtypes)

forecaster.predict(steps=10)

{'item_1': 0, 'item_2': 1, 'item_3': 2}
['_level_skforecast']
lag_1                 float64
lag_2                 float64
lag_3                 float64
lag_4                 float64
lag_5                 float64
lag_6                 float64
lag_7                 float64
lag_8                 float64
lag_9                 float64
lag_10                float64
lag_11                float64
lag_12                float64
lag_13                float64
lag_14                float64
lag_15                float64
lag_16                float64
lag_17                float64
lag_18                float64
lag_19                float64
lag_20                float64
lag_21                float64
lag_22                float64
lag_23                float64
lag_24                float64
_level_skforecast    category
dtype: object


,item_1,item_2,item_3
2014-07-16,25.792675,11.173958,12.077978
2014-07-17,25.546403,10.943413,10.099237
2014-07-18,25.233413,11.594754,12.391873
2014-07-19,23.950772,11.603061,12.225715
2014-07-20,21.515123,11.582226,11.183190
2014-07-21,22.310733,11.005379,10.950110
2014-07-22,24.708569,11.527696,10.707694
2014-07-23,25.092788,11.724573,12.762186
2014-07-24,25.394611,10.466541,11.711561
2014-07-25,25.516455,10.938599,11.257149


In [ ]:
# Series weights

forecaster = ForecasterAutoregMultiSeries(
                 regressor          = HistGradientBoostingRegressor(random_state=123, categorical_features=['_level_skforecast']),
                 lags               = 24,
                 encoding           = 'ordinal_category',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = {'item_1': 0, 'item_2': 1, 'item_3': 2},
                 forecaster_id      = None
             )

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
sample_weights = forecaster.create_sample_weights(series_col_names= series_col_names, X_train =  X_train)

results = X_train[['_level_skforecast']].assign(sample_weights = sample_weights)
(results['_level_skforecast'] == results['sample_weights']).all()

True

In [ ]:

forecaster = ForecasterAutoregMultiSeries(
                 regressor          = HistGradientBoostingRegressor(random_state=123, categorical_features=['_level_skforecast']),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = {'item_1': 0, 'item_2': 1, 'item_3': 2},
                 forecaster_id      = None
             )

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
sample_weights = forecaster.create_sample_weights(series_col_names= series_col_names, X_train =  X_train)

results = X_train[['_level_skforecast']].assign(sample_weights = sample_weights)
(results['_level_skforecast'] == results['sample_weights']).all()

In [ ]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = HistGradientBoostingRegressor(random_state=123, categorical_features=['_level_skforecast']),
                 lags               = 24,
                 encoding           = 'onehot',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = {'item_1': 0, 'item_2': 1, 'item_3': 2},
                 forecaster_id      = None
             )

X_train, y_train, series_indexes, series_col_names, exog_col_names, exog_dtypes = forecaster.create_train_X_y(data_train)
sample_weights = forecaster.create_sample_weights(series_col_names= series_col_names, X_train =  X_train)

results = X_train[['_level_skforecast']].assign(sample_weights = sample_weights)
(results['_level_skforecast'] == results['sample_weights']).all()

In [7]:
# Unit test _create_train_X_y ForecasterAutoregMultiSeries
# ==============================================================================
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.exceptions import MissingValuesWarning
from skforecast.exceptions import IgnoredArgumentWarning
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [16]:
def test_create_train_X_y_output_when_series_and_exog_is_None_ordinal_encoding():
    """
    Test the output of _create_train_X_y when series has 2 columns and 
    exog is None.
    """
    series = pd.DataFrame({'1': pd.Series(np.arange(7, dtype=float)), 
                           '2': pd.Series(np.arange(7, dtype=float))})
    forecaster = ForecasterAutoregMultiSeries(
        LinearRegression(),
        lags=3,
        encoding="ordinal_category"
    )

    results = forecaster._create_train_X_y(series=series)
    expected = (
        pd.DataFrame(
            data = np.array([[-0.5, -1. , -1.5, 0.],
                             [ 0. , -0.5, -1. , 0.],
                             [ 0.5,  0. , -0.5, 0.],
                             [ 1. ,  0.5,  0. , 0.],
                             [-0.5, -1. , -1.5, 1.],
                             [ 0. , -0.5, -1. , 1.],
                             [ 0.5,  0. , -0.5, 1.],
                             [ 1. ,  0.5,  0. , 1.]]),
            index   = pd.Index([3, 4, 5, 6, 3, 4, 5, 6]),
            columns = ['lag_1', 'lag_2', 'lag_3', '_level_skforecast'],
        ).astype({'_level_skforecast': int}).astype({'_level_skforecast': 'category'}),
        pd.Series(
            data  = np.array([0., 0.5, 1., 1.5, 0., 0.5, 1., 1.5]),
            index = pd.Index([3, 4, 5, 6, 3, 4, 5, 6]),
            name  = 'y',
            dtype = float
        ),
        {'1': pd.RangeIndex(start=0, stop=7, step=1),
         '2': pd.RangeIndex(start=0, stop=7, step=1)},
        ['1', '2'],
        None,
        None,
        {'1': pd.Series(
                  data  = np.array([4., 5., 6.]),
                  index = pd.RangeIndex(start=4, stop=7, step=1),
                  name  = '1',
                  dtype = float
              ),
         '2': pd.Series(
                  data  = np.array([4., 5., 6.]),
                  index = pd.RangeIndex(start=4, stop=7, step=1),
                  name  = '2',
                  dtype = float
              )
        }
    )

    pd.testing.assert_frame_equal(results[0], expected[0])
    pd.testing.assert_series_equal(results[1], expected[1])
    for k in results[2].keys():
        pd.testing.assert_index_equal(results[2][k], expected[2][k])
    assert results[3] == expected[3]
    assert isinstance(results[4], type(None))
    assert isinstance(results[5], type(None))
    for k in results[6].keys():
        pd.testing.assert_series_equal(results[6][k], expected[6][k])

test_create_train_X_y_output_when_series_and_exog_is_None_ordinal_encoding()

In [12]:
def test_create_train_X_y_output_when_series_and_exog_no_pandas_index_ordinal_encoding():
    """
    Test the output of _create_train_X_y when series and exog have no pandas index 
    that doesn't start at 0.
    """
    series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                           'l2': pd.Series(np.arange(10, dtype=float))})
    series.index = np.arange(6, 16)
    exog = pd.Series(np.arange(100, 110), index=np.arange(6, 16), 
                     name='exog', dtype=float)
    
    forecaster = ForecasterAutoregMultiSeries(
                    LinearRegression(),
                    lags=5,
                    encoding=encoding,
                    transformer_series=None
                 )
    
    warn_msg = re.escape(
        ("Series {'l3'} are not present in `series`. No last window is stored for them.")
    )
    with pytest.warns(IgnoredArgumentWarning, match = warn_msg): 
        results = forecaster._create_train_X_y(series=series, exog=exog,
                                               store_last_window=['l1', 'l2', 'l3'])

    expected = (
        pd.DataFrame(
            data = np.array([[4., 3., 2., 1., 0., 0., 105.],
                             [5., 4., 3., 2., 1., 0., 106.],
                             [6., 5., 4., 3., 2., 0., 107.],
                             [7., 6., 5., 4., 3., 0., 108.],
                             [8., 7., 6., 5., 4., 0., 109.],
                             [4., 3., 2., 1., 0., 1., 105.],
                             [5., 4., 3., 2., 1., 1., 106.],
                             [6., 5., 4., 3., 2., 1., 107.],
                             [7., 6., 5., 4., 3., 1., 108.],
                             [8., 7., 6., 5., 4., 1., 109.]]),
            index   = pd.Index([5, 6, 7, 8, 9, 5, 6, 7, 8, 9]),
            columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 
                       '_level_skforecast', 'exog']
        ).astype({'_level_skforecast': int}).astype({'_level_skforecast': dtype}),
        pd.Series(
            data  = np.array([5, 6, 7, 8, 9, 5, 6, 7, 8, 9]),
            index = pd.Index([5, 6, 7, 8, 9, 5, 6, 7, 8, 9]),
            name  = 'y',
            dtype = float
        ),
        {'l1': pd.RangeIndex(start=0, stop=10, step=1),
         'l2': pd.RangeIndex(start=0, stop=10, step=1)},
        ['l1', 'l2'],
        ['exog'],
        {'exog': exog.dtypes},
        {'l1': pd.Series(
                   data  = np.array([5., 6., 7., 8., 9.]),
                   index = pd.RangeIndex(start=5, stop=10, step=1),
                   name  = 'l1',
                   dtype = float
               ),
         'l2': pd.Series(
                   data  = np.array([5., 6., 7., 8., 9.]),
                   index = pd.RangeIndex(start=5, stop=10, step=1),
                   name  = 'l2',
                   dtype = float
               )
        }
    )

    pd.testing.assert_frame_equal(results[0], expected[0])
    pd.testing.assert_series_equal(results[1], expected[1])
    for k in results[2].keys():
        pd.testing.assert_index_equal(results[2][k], expected[2][k])
    assert results[3] == expected[3]
    assert set(results[4]) == set(expected[4])
    for k in results[5].keys():
        assert results[5] == expected[5]
    for k in results[6].keys():
        pd.testing.assert_series_equal(results[6][k], expected[6][k])

,lag_1,lag_2,lag_3,_level_skforecast
3,-0.5,-1.0,-1.5,0
4,0.0,-0.5,-1.0,0
5,0.5,0.0,-0.5,0
6,1.0,0.5,0.0,0
3,-0.5,-1.0,-1.5,1
4,0.0,-0.5,-1.0,1
5,0.5,0.0,-0.5,1
6,1.0,0.5,0.0,1


In [22]:
series = {
    'l1': pd.Series(np.arange(10, dtype=float)), 
    'l2': pd.Series(np.arange(15, 20, dtype=float)),
    'l3': pd.Series(np.arange(20, 25, dtype=float))
}
series['l1'].loc[3] = np.nan
series['l2'].loc[2] = np.nan
series['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
series['l2'].index = pd.date_range("1990-01-05", periods=5, freq='D')
series['l3'].index = pd.date_range("1990-01-03", periods=5, freq='D')
series

{'l1': 1990-01-01    0.0
 1990-01-02    1.0
 1990-01-03    2.0
 1990-01-04    NaN
 1990-01-05    4.0
 1990-01-06    5.0
 1990-01-07    6.0
 1990-01-08    7.0
 1990-01-09    8.0
 1990-01-10    9.0
 Freq: D, dtype: float64,
 'l2': 1990-01-05    15.0
 1990-01-06    16.0
 1990-01-07     NaN
 1990-01-08    18.0
 1990-01-09    19.0
 Freq: D, dtype: float64,
 'l3': 1990-01-03    20.0
 1990-01-04    21.0
 1990-01-05    22.0
 1990-01-06    23.0
 1990-01-07    24.0
 Freq: D, dtype: float64}

In [31]:
pd.DataFrame(
            data = np.array(
                       [[np.nan, 2.    , 1.    , 0, 104.  , np.nan],
                        [4.    , np.nan, 2.    , 0, 105.  , np.nan],
                        [5.    , 4.    , np.nan, 0, 106.  , np.nan],
                        [6.    , 5.    , 4.    , 0, 107.  , np.nan],
                        [7.    , 6.    , 5.    , 0, 108.  , np.nan],
                        [8.    , 7.    , 6.    , 0, 109.  , np.nan],
                        [np.nan, 16.   , 15.   , 1, np.nan, np.nan],
                        [18.   , np.nan, 16.   , 1, np.nan, np.nan],
                        [22.   , 21.   , 20.   , 2, 206.  , 'b'   ],
                        [23.   , 22.   , 21.   , 2, np.nan, np.nan]]
                    ),
            index   = pd.Index(
                          pd.DatetimeIndex(
                              ['1990-01-05', '1990-01-06', '1990-01-07', '1990-01-08', '1990-01-09', '1990-01-10',
                               '1990-01-08', '1990-01-09', 
                               '1990-01-06', '1990-01-07']
                          )
                      ),
            columns = ['lag_1', 'lag_2', 'lag_3', '_level_skforecast', 
                       'exog_1', 'exog_2']
        ).astype({'lag_1': float, 'lag_2': float, 'lag_3': float, '_level_skforecast': int,
                  'exog_1': float, 'exog_2': object}
        ).astype({'_level_skforecast': int})

,lag_1,lag_2,lag_3,_level_skforecast,exog_1,exog_2
1990-01-05,NaN,2.0,1.0,0,104.0,nan
1990-01-06,4.0,NaN,2.0,0,105.0,nan
1990-01-07,5.0,4.0,NaN,0,106.0,nan
1990-01-08,6.0,5.0,4.0,0,107.0,nan
1990-01-09,7.0,6.0,5.0,0,108.0,nan
1990-01-10,8.0,7.0,6.0,0,109.0,nan
1990-01-08,NaN,16.0,15.0,1,NaN,nan
1990-01-09,18.0,NaN,16.0,1,NaN,nan
1990-01-06,22.0,21.0,20.0,2,206.0,b
1990-01-07,23.0,22.0,21.0,2,NaN,nan


In [38]:
series = pd.DataFrame({'1': pd.Series(np.arange(7, dtype=float)), 
                           '2': pd.Series(np.arange(7, dtype=float))})
forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(),
    lags=3,
    encoding="onehot"
)
_ = forecaster._create_train_X_y(series=series)
forecaster.encoding_mapping

{'1': 0, '2': 1}

In [36]:
def test_create_train_X_y_encoding_mapping():
    """
    Test the encoding mapping of _create_train_X_y.
    """
    series = pd.DataFrame({'1': pd.Series(np.arange(7, dtype=float)), 
                           '2': pd.Series(np.arange(7, dtype=float))})
    forecaster = ForecasterAutoregMultiSeries(
        LinearRegression(),
        lags=3,
        encoding="ordinal"
    )
    _ = forecaster._create_train_X_y(series=series)
    
    assert forecaster.encoding_mapping == {'1': 0, '2': 1}